<a href="https://colab.research.google.com/github/stendewa/Stock-Audit-Automation-With-Odoo-Data/blob/master/Stock_Audit_Forensic_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries for data analysis
!pip install pandas openpyxl


In [ ]:
# Import pandas for structured data manipulation
import pandas as pd

# Import numpy for numerical calculations
import numpy as np


In [ ]:
# Import the drive module from Google Colab to connect Google Drive
from google.colab import drive

# Mount Google Drive to the Colab environment
drive.mount('/content/drive')

# Define the full path to your products file inside Google Drive
# Replace 'MyDrive/YourFolder/' with your actual folder structure
products_path = "/content/drive/MyDrive/YourFolder/products.xlsx"

# Define the full path to your product moves file inside Google Drive
moves_path = "/content/drive/MyDrive/YourFolder/product_moves.xlsx"

# Load the products list from Google Drive into a pandas DataFrame
products = pd.read_excel(products_path)

# Load the product moves file from Google Drive into a pandas DataFrame
movements = pd.read_excel(moves_path)


In [ ]:
# Remove leading/trailing spaces in product names
products["Name"] = products["Name"].astype(str).str.strip()

# Remove leading/trailing spaces in movement product column
movements["Product"] = movements["Product"].astype(str).str.strip()

# Ensure numeric columns are numeric
products["Quantity On Hand"] = pd.to_numeric(products["Quantity On Hand"], errors="coerce").fillna(0)
products["Cost"] = pd.to_numeric(products["Cost"], errors="coerce").fillna(0)

# Ensure movement quantity is numeric
movements["Quantity"] = pd.to_numeric(movements["Quantity"], errors="coerce").fillna(0)

# Filter only completed movements
movements = movements[movements["Status"] == "Done"]



In [ ]:
# Define your main physical location
shop_location = "SHOP/Stock"

# Create signed quantity column
movements["Signed_Qty"] = np.where(
    movements["To"] == shop_location,           # If stock moved INTO shop
    movements["Quantity"],                      # Add quantity
    np.where(
        movements["From"] == shop_location,     # If stock moved OUT of shop
        -movements["Quantity"],                 # Subtract quantity
        0                                   # Ignore movements not affecting shop
    )
)


In [ ]:
# Group by product and sum signed quantities
movement_totals = movements.groupby("Product")["Signed_Qty"].sum().reset_index()

# Rename column for clarity
movement_totals.rename(columns={"Signed_Qty": "Reconstructed_Stock"}, inplace=True)


In [ ]:
# Merge movement totals with product master
df = products.merge(
    movement_totals,
    left_on="Name",
    right_on="Product",
    how="left"
)

# Replace missing reconstructed values with 0
df["Reconstructed_Stock"] = df["Reconstructed_Stock"].fillna(0)


In [ ]:
# Calculate variance between Odoo and reconstructed movement
df["Variance"] = df["Quantity On Hand"] - df["Reconstructed_Stock"]

# Calculate financial exposure
df["Value_Impact"] = df["Variance"] * df["Cost"]


In [ ]:
# Filter missing items
missing_items = df[
    (df["Quantity On Hand"] == 0) &
    (df["Reconstructed_Stock"] > 0)
]

# Sort by highest value loss
missing_items = missing_items.sort_values(by="Value_Impact", ascending=False)

missing_items.head()


,Favorite,Name,Internal Reference,Sales Price,Cost,Quantity On Hand,Forecasted Quantity,Activity Exception Decoration,Product,Reconstructed_Stock,Variance,Value_Impact
590,False,Galaxy Tab S11 Cover,NaN,1,0.0,0,0,NaN,Galaxy Tab S11 Cover,1.0,-1.0,-0.0
481,False,Earldom Car Phone Mount EH153,NaN,1,0.0,0,0,NaN,Earldom Car Phone Mount EH153,1.0,-1.0,-0.0
664,False,Green Lion Airpods Pro 3 case,NaN,1,0.0,0,0,NaN,Green Lion Airpods Pro 3 case,1.0,-1.0,-0.0
642,False,Google Nest Mini,NaN,1,0.0,0,0,NaN,Google Nest Mini,1.0,-1.0,-0.0
2911,False,Spigen Airpods Pro 2 Case,NaN,1,0.0,0,0,NaN,Spigen Airpods Pro 2 Case,1.0,-1.0,-0.0


In [ ]:
# Detect overstated inventory
overstated = df[
    df["Quantity On Hand"] > df["Reconstructed_Stock"]
]

overstated.sort_values(by="Value_Impact", ascending=False).head()


,Favorite,Name,Internal Reference,Sales Price,Cost,Quantity On Hand,Forecasted Quantity,Activity Exception Decoration,Product,Reconstructed_Stock,Variance,Value_Impact
2937,False,Starlink Mini Kit,NaN,44000,37800.0,0,-14,NaN,Starlink Mini Kit,-64.0,64.0,2419200.0
1088,False,Iphone 17 Pro Max 256Gb,NaN,210000,187000.0,0,-6,NaN,Iphone 17 Pro Max 256Gb,-3.0,3.0,561000.0
3403,False,iPad A16 128GB A3354,NaN,68000,498000.0,0,-1,NaN,iPad A16 128GB A3354,-1.0,1.0,498000.0
1053,False,Iphone 15 Pro Max 256GB,NaN,115000,105000.0,0,0,NaN,Iphone 15 Pro Max 256GB,-3.0,3.0,315000.0
2519,False,SAMSUNG GALAXY Z FOLD 6,NaN,170000,140000.0,0,1,NaN,SAMSUNG GALAXY Z FOLD 6,-2.0,2.0,280000.0


In [ ]:
# Flag serious variance
df["Flag"] = np.where(
    abs(df["Variance"]) > 1,   # Adjust tolerance if needed
    "INVESTIGATE",
    "OK"
)

issues = df[df["Flag"] == "INVESTIGATE"]

issues.sort_values(by="Value_Impact", ascending=False).head()


,Favorite,Name,Internal Reference,Sales Price,Cost,Quantity On Hand,Forecasted Quantity,Activity Exception Decoration,Product,Reconstructed_Stock,Variance,Value_Impact,Flag
2937,False,Starlink Mini Kit,NaN,44000,37800.0,0,-14,NaN,Starlink Mini Kit,-64.0,64.0,2419200.0,INVESTIGATE
1088,False,Iphone 17 Pro Max 256Gb,NaN,210000,187000.0,0,-6,NaN,Iphone 17 Pro Max 256Gb,-3.0,3.0,561000.0,INVESTIGATE
1053,False,Iphone 15 Pro Max 256GB,NaN,115000,105000.0,0,0,NaN,Iphone 15 Pro Max 256GB,-3.0,3.0,315000.0,INVESTIGATE
890,False,Hp EliteBook 1040 G10 i7 16/512Gb,NaN,149000,140000.0,0,1,NaN,Hp EliteBook 1040 G10 i7 16/512Gb,-2.0,2.0,280000.0,INVESTIGATE
2519,False,SAMSUNG GALAXY Z FOLD 6,NaN,170000,140000.0,0,1,NaN,SAMSUNG GALAXY Z FOLD 6,-2.0,2.0,280000.0,INVESTIGATE


In [ ]:
# Identify adjustment references (edit keyword if needed)
adjustments = movements[movements["Reference"].str.contains("ADJ", case=False, na=False)]

# Sum adjustment impact per product
adjustment_totals = adjustments.groupby("Product")["Signed_Qty"].sum().reset_index()

# Sort worst offenders
adjustment_totals.sort_values(by="Signed_Qty").head()


,Product,Signed_Qty


In [ ]:
# Export full audit sheet
df.to_excel("Full_Stock_Audit.xlsx", index=False)

# Export missing items
missing_items.to_excel("Missing_Items.xlsx", index=False)

# Export major discrepancies
issues.to_excel("Variance_Issues.xlsx", index=False)


In [ ]:
# Filter internal transfers (not vendor movements)
internal_transfers = movements[
    (movements["From"].str.contains("SHOP", na=False)) &
    (~movements["To"].str.contains("Vendor", na=False))
]


In [ ]:
# Create location pair column
internal_transfers["Route"] = internal_transfers["From"] + " → " + internal_transfers["To"]

# Aggregate quantities per route
route_totals = internal_transfers.groupby(["Product", "Route"])["Quantity"].sum().reset_index()

route_totals.sort_values(by="Quantity", ascending=False).head()


/tmp/ipython-input-2210913474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internal_transfers["Route"] = internal_transfers["From"] + " → " + internal_transfers["To"]


,Product,Route,Quantity
912,PS4 Pad Copy,SHOP/Stock → Partners/Customers,122
1046,Phone back cover FAA,SHOP/Stock → Virtual Locations/Inventory adjus...,97
1420,Starlink Mini Kit,SHOP/Stock → Partners/Customers,81
915,PS4 Pad Copy Preowned,SHOP/Stock → Virtual Locations/Inventory adjus...,79
970,PS5 FC26 (DUBAI),SHOP/Stock → Partners/Customers,62


In [ ]:
# Stock leaving shop
leaving_shop = movements[movements["From"] == shop_location].groupby("Product")["Quantity"].sum().reset_index()
leaving_shop.rename(columns={"Quantity": "Total_Leaving"}, inplace=True)

# Stock returning to shop
returning_shop = movements[movements["To"] == shop_location].groupby("Product")["Quantity"].sum().reset_index()
returning_shop.rename(columns={"Quantity": "Total_Returning"}, inplace=True)

# Merge
transfer_balance = leaving_shop.merge(returning_shop, on="Product", how="left")
transfer_balance["Total_Returning"] = transfer_balance["Total_Returning"].fillna(0)

# Calculate imbalance
transfer_balance["Transfer_Imbalance"] = transfer_balance["Total_Leaving"] - transfer_balance["Total_Returning"]

# Flag major imbalance
transfer_leakage = transfer_balance[transfer_balance["Transfer_Imbalance"] > 2]

transfer_leakage.sort_values(by="Transfer_Imbalance", ascending=False).head()


,Product,Total_Leaving,Total_Returning,Transfer_Imbalance
787,PS4 Pad Copy,122,37.0,85.0
1243,Starlink Mini Kit,81,17.0,64.0
906,Phone back cover BAA,10,0.0,10.0
721,PS3 Pad,8,1.0,7.0
1471,[R3CW20B1S9T] Samsung Galaxy S23 Ultra Preowne...,9,4.0,5.0


In [ ]:
# Create risk score column
df["Risk_Score"] = 0

# Add risk points for variance
df.loc[abs(df["Variance"]) > 1, "Risk_Score"] += 2

# Add risk for high value variance
df.loc[abs(df["Value_Impact"]) > 10000, "Risk_Score"] += 3

# Add risk for transfer imbalance
df = df.merge(
    transfer_balance[["Product", "Transfer_Imbalance"]],
    left_on="Name",
    right_on="Product",
    how="left"
)

df["Transfer_Imbalance"] = df["Transfer_Imbalance"].fillna(0)

df.loc[df["Transfer_Imbalance"] > 2, "Risk_Score"] += 2

df.sort_values(by="Risk_Score", ascending=False).head()



,Favorite,Name,Internal Reference,Sales Price,Cost,Quantity On Hand,Forecasted Quantity,Activity Exception Decoration,Product_x,Reconstructed_Stock,Variance,Value_Impact,Flag,Risk_Score,Product_y,Transfer_Imbalance
1993,False,PS5 FC26 LOCAL,NaN,6500,5500.0,0,-5,NaN,PS5 FC26 LOCAL,-3.0,3.0,16500.0,INVESTIGATE,7,PS5 FC26 LOCAL,3.0
1053,False,Iphone 15 Pro Max 256GB,NaN,115000,105000.0,0,0,NaN,Iphone 15 Pro Max 256GB,-3.0,3.0,315000.0,INVESTIGATE,7,Iphone 15 Pro Max 256GB,3.0
2630,False,Samsung Galaxy Fit 3 Black,NaN,5000,4500.0,0,-4,NaN,Samsung Galaxy Fit 3 Black,-3.0,3.0,13500.0,INVESTIGATE,7,Samsung Galaxy Fit 3 Black,3.0
2937,False,Starlink Mini Kit,NaN,44000,37800.0,0,-14,NaN,Starlink Mini Kit,-64.0,64.0,2419200.0,INVESTIGATE,7,Starlink Mini Kit,64.0
1155,False,Jbl Wireless Dual Microphones,NaN,12000,11000.0,0,-2,NaN,Jbl Wireless Dual Microphones,-3.0,3.0,33000.0,INVESTIGATE,7,Jbl Wireless Dual Microphones,3.0
